<a href="https://colab.research.google.com/github/Ankur3107/colab_notebooks/blob/master/Generic_Transformer_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install transformers

In [13]:
import os, pandas as pd
from sklearn.model_selection import train_test_split
import logging
from transformers import *
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

# 1. Set Configuration

In [14]:
class Config:
  train_file = './data.csv'
  eval_file = './eval.csv'
  max_seq_len = 128
  batch_size = 32
  epochs = 5
  model_name = 'bert-base-uncased'
  learning_rate = 2e-5
  n_classes = 3
  device = 'cpu'
  


flags = Config

# 2. Build Dataset Pipeline

In [15]:
class TextLabelDataset(Dataset):

    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
  
    def __len__(self):
        return len(self.texts)
  
    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          return_token_type_ids=False,
          pad_to_max_length=True,
          return_attention_mask=True,
          return_tensors='pt',
        )

        return {
          'texts': text,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'targets': torch.tensor(label, dtype=torch.long)
        }

def create_data_loader(df, tokenizer, max_len, batch_size, is_prediction=False):

  if isinstance(df, str):
    df = pd.read_csv(df)
  else:
    pass

  if is_prediction:
    ds = TextLabelDataset(
        texts=df.text.to_numpy(),
        labels=np.array([-1]*len(df.text.values)),
        tokenizer=tokenizer,
        max_len=max_len
        )
  else:
    ds = TextLabelDataset(
        texts=df.text.to_numpy(),
        labels=df.labels.to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
        )

    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=4
        )

# 3. Build Model 

In [16]:
class Classifier(nn.Module):

  def __init__(self, model_name, n_classes):
      super(Classifier, self).__init__()
      self.bert = AutoModel.from_pretrained(model_name)
      self.drop = nn.Dropout(p=0.3)
      self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask):
      _, pooled_output = self.bert(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      output = self.drop(pooled_output)
      return self.out(output)

In [ ]:
class ClassificationModel:

  def __init__(self, flags):
    self.flags = flags
    self.tokenizer = BertTokenizer.from_pretrained(self.flags.model_name)
    self.model = Classifier(self.flags.model_name, self.flags.n_classes)
    self.model = self.model.to(self.flags.device)

  def train():

    train_data_loader = create_data_loader(self.flags.train_file, self.okenizer, self.flags.max_seq_len, self.flags.batch_size)
    val_data_loader = create_data_loader(self.flags.eval_file, self.tokenizer, self.flags.max_seq_len, self.flags.batch_size)

    optimizer = AdamW(self.model.parameters(), lr=self.flags.learning_rate, correct_bias=False)
    total_steps = len(train_data_loader) * self.flags.epochs

    scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps=0,
      num_training_steps=total_steps
    )

    loss_fn = nn.CrossEntropyLoss().to(self.flags.device)

    history = defaultdict(list)
    best_accuracy = 0

    for epoch in self.flags.epochs:

      print(f'Epoch {epoch + 1}/{EPOCHS}')
      print('-' * 10)

      train_acc, train_loss = self.train_epoch(
        self.model,
        train_data_loader,    
        loss_fn, 
        optimizer, 
        self.flags.device, 
        scheduler, 
        len(train_df)
      )

      print(f'Train loss {train_loss} accuracy {train_acc}')

      val_acc, val_loss = eval_model(
        classifier_model,
        val_data_loader,
        loss_fn, 
        device, 
        len(eval_df)
      )

      print(f'Val   loss {val_loss} accuracy {val_acc}')
      print()

      history['train_acc'].append(train_acc)
      history['train_loss'].append(train_loss)
      history['val_acc'].append(val_acc)
      history['val_loss'].append(val_loss)

      if val_acc > best_accuracy:
        torch.save(classifier_model.state_dict(), 'best_model_state.bin')
        best_accuracy = val_acc


  def train_epoch(self, model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model = model.train()

    losses = []
    correct_predictions = 0

    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )

      _, preds = torch.max(outputs, dim=1)
      loss = loss_fn(outputs, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

      loss.backward()
      nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
      optimizer.step()
      scheduler.step()
      optimizer.zero_grad()

    return correct_predictions.double() / n_examples, np.mean(losses)

  def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()

    losses = []
    correct_predictions = 0

    with torch.no_grad():
      for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)

        outputs = model(
          input_ids=input_ids,
          attention_mask=attention_mask
        )
        _, preds = torch.max(outputs, dim=1)

        loss = loss_fn(outputs, targets)

        correct_predictions += torch.sum(preds == targets)
        losses.append(loss.item())

    return correct_predictions.double() / n_examples, np.mean(losses)


    

In [ ]:
EPOCHS = 5

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)